<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

#  Summary
This notebook looks through the reddit whisky reviews and searches on vintages.com for the whisky name and returns any results. The results of the search are recorded into parquet files depending on whether they have no match, one match ,or multiple matches.

At the moment all I am looking for is the product ID so we can pull product details from the lcbo api in the next step.

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.parse
import urllib.request
import re
import time
from urllib.error import HTTPError

In [ ]:
rdb = pd.read_excel('redditwhiskeydatabase.xlsx')

In [ ]:
rdb.head()

In [ ]:
uniquewhiskies = rdb.assign(Whisky_Name = lambda x: x['Whisky Name'].str.strip())['Whisky_Name'].unique()

In [ ]:
print(rdb.shape)
print(uniquewhiskies.shape)

In [ ]:
uniquewhiskies = pd.DataFrame(uniquewhiskies, columns=['RedditWhiskyName']).reset_index().rename({'index':'RedditWhiskyID'},axis='columns')

In [ ]:
def productSearch(itemname):
    # strip out spaces and non ascii characters
    itemname = re.sub(r'[^\x00-\x7F]+',' ',  itemname.replace(' ','%20'))
    url = 'http://www.vintages.com/lcbo-ear/vintages/product/searchResults.do?ITEM_NAME=' + itemname.replace(' ','%20')
    user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
    values = {'name' : 'Firstname Lastname',
              'location' : 'Toronto',
              'language' : 'Python' }
    headers = { 'User-Agent' : user_agent }
    print(url)
    data = urllib.parse.urlencode(values)
    data = data.encode('ascii')
    req = urllib.request.Request(url, data, headers)
    try:
        with urllib.request.urlopen(req) as response:
           content = response.read()
        return BeautifulSoup(content, 'html.parser')
    except :
        return None

In [ ]:
def getProductIds(itemname):
    result = productSearch(itemname)
    if result is None:
        return None
    matches = pd.DataFrame(columns=['Name','ProductID'])
    if bool(re.search('Search',str(result.title))):
        # More than one result
        tables = result.findAll("table", {"class": "search"})    
        for table in tables:
            row = table.findAll('a',href=True)
            if len(row) == 0:
                return None
            else:
                link = row[1]['href']
                name = row[1].text.strip()
                productid = re.search('(?<=itemNumber\=)[0-9]*', link)[0]
                matches = matches.append(pd.DataFrame([[name,productid]], columns=['Name','ProductID']),ignore_index=True)
    else: 
        # Only one result
        name = result.find("input",{"name":"itemName"})['value']
        productid = result.find("input",{"name":"itemNumber"})['value']
        matches = matches.append(pd.DataFrame([[name,productid]], columns=['Name','ProductID']),ignore_index=True)
    
    return matches

In [ ]:
uniquewhiskies.iloc[2500]

In [ ]:
nomatch = pd.DataFrame(columns=['RedditWhiskyID','RedditWhiskyName'])
onematch = pd.DataFrame(columns=['RedditWhiskyID','RedditWhiskyName','Name','ProductID'])
product_ids = pd.DataFrame(columns=['RedditWhiskyID','RedditWhiskyName','Name','ProductID'])
for whisky in uniquewhiskies.itertuples():
    myId = getProductIds(whisky.RedditWhiskyName)
    time.sleep(3)
    if myId is None:
        nomatch = nomatch.append(pd.DataFrame([[whisky.RedditWhiskyID,whisky.RedditWhiskyName]],
                                              columns=['RedditWhiskyID','RedditWhiskyName']))
    elif len(myId) == 0:
        nomatch = nomatch.append(pd.DataFrame([[whisky.RedditWhiskyID,whisky.RedditWhiskyName]],
                                              columns=['RedditWhiskyID','RedditWhiskyName']))
    elif len(myId) == 1:
        myId['RedditWhiskyID'] = whisky.RedditWhiskyID
        myId['RedditWhiskyName'] = whisky.RedditWhiskyName
        onematch = onematch.append(myId)
    else:
        myId['RedditWhiskyID'] = whisky.RedditWhiskyID
        myId['RedditWhiskyName'] = whisky.RedditWhiskyName
        product_ids = product_ids.append(myId)

continuing where we left off:


In [ ]:
nomatch.to_parquet('nomatch.parquet')
onematch.to_parquet('onematch.parquet')
product_ids.to_parquet('product_ids.parquet')

In [ ]:
nomatch.shape

In [ ]:
onematch.shape

In [ ]:
product_ids.shape

In [ ]:
uniquewhiskies.shape

In [ ]:
product_ids